# Episode 19. Supervised Learning For Cross-Sectional Analysis

## Data: Boston Housing Prices
### Keras Built-in function - https://keras.io/api/datasets/boston_housing/
### Excel file form Kaggle - https://www.kaggle.com/fedesoriano/the-boston-houseprice-data


In [ ]:
### Original Data - http://lib.stat.cmu.edu/datasets/boston
#### The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
#### prices and the demand for clean air', J. Environ. Economics & Management,
#### vol.5, 81-102, 1978.

# There are 14 attributes in each case of the dataset. They are:
# CRIM - per capita crime rate by town
# ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
# INDUS - proportion of non-retail business acres per town.
# CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
# NOX - nitric oxides concentration (parts per 10 million)
# RM - average number of rooms per dwelling
# AGE - proportion of owner-occupied units built prior to 1940
# DIS - weighted distances to five Boston employment centres
# RAD - index of accessibility to radial highways
# TAX - full-value property-tax rate per $10,000
# PTRATIO - pupil-teacher ratio by town
# B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
# LSTAT - % lower status of the population
# MEDV - Median value of owner-occupied homes in $1000's

## Step 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Step 2. Load Data

In [ ]:
# Load Loan Default Data
# Original Data Source: https://keras.io/api/datasets/boston_housing
from google.colab import files
uploaded = files.upload()

Saving boston.csv to boston (2).csv


In [ ]:
# Store data in DataFrame
df = pd.read_csv("boston.csv")

## Step 3. Check & Cleanup Data

In [ ]:
# check data
print(df.keys())
# rename columns: small letter, remove special characters and space
df.columns = df.columns.str.lower()
# check modified column names
print(df.keys())

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')
Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat', 'medv'],
      dtype='object')


In [ ]:
# number of missing data by columns
print(df.isna().sum())
# drop missing data
drop_df = df.dropna(how='any')
print(drop_df.shape)

crim       0
zn         0
indus      0
chas       0
nox        0
rm         0
age        0
dis        0
rad        0
tax        0
ptratio    0
b          0
lstat      0
medv       0
dtype: int64
(506, 14)


In [ ]:
# Find Duplicate Data
duplicate_rows_df = df[df.duplicated()]
print(duplicate_rows_df.shape)
# drop duplicate data
print(df.drop_duplicates().shape) 

(0, 14)
(506, 14)


In [ ]:
# summary statistics
df.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## Step 4. Set Training Variables

In [ ]:
# split train & test data
from sklearn.model_selection import train_test_split
x = df.loc[:, df.columns != 'medv']
y = df.loc[:, df.columns == 'medv']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)

In [ ]:
# check train & test data
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(354, 13)
(354, 1)
(152, 13)
(152, 1)


In [ ]:
# summary stats
x_train.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
count,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000
mean,3.515843,11.432203,10.889379,0.059322,0.550042,6.314153,67.660734,3.853660,9.214689,401.550847,18.441525,360.450537,12.592599
std,9.203000,22.989767,6.839935,0.236561,0.113865,0.708672,28.354127,2.127925,8.451106,166.332065,2.133292,87.909480,7.222552
min,0.013010,0.000000,0.460000,0.000000,0.392000,3.561000,6.000000,1.169100,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.080573,0.000000,5.145000,0.000000,0.448250,5.926500,42.325000,2.113700,4.000000,277.000000,17.400000,377.255000,6.877500
50%,0.223250,0.000000,8.560000,0.000000,0.524000,6.214000,76.500000,3.294800,5.000000,329.000000,18.950000,392.215000,10.805000
75%,2.662860,19.375000,18.100000,0.000000,0.624000,6.634000,92.975000,5.226975,8.000000,666.000000,20.200000,396.295000,17.117500
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.725000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000


In [ ]:
# standardize data
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

In [ ]:
# check mean
print(x_train.mean(axis=0))
print(x_test.mean(axis=0))

crim      -5.139486e-17
zn        -6.742880e-17
indus      3.329415e-15
chas      -3.362031e-16
nox        1.761303e-15
rm         2.008437e-15
age        4.522434e-16
dis       -5.582477e-16
rad       -5.394304e-17
tax       -3.261672e-17
ptratio    1.712378e-14
b          7.638232e-15
lstat      7.849967e-16
dtype: float64
crim       0.035334
zn        -0.009929
indus      0.120408
chas       0.138583
nox        0.136033
rm        -0.138660
age        0.107329
dis       -0.091702
rad        0.131848
tax        0.133819
ptratio    0.021859
b         -0.143008
lstat      0.027869
dtype: float64


In [ ]:
# check std
print(x_train.std(axis=0))
print(x_test.std(axis=0))

crim       1.0
zn         1.0
indus      1.0
chas       1.0
nox        1.0
rm         1.0
age        1.0
dis        1.0
rad        1.0
tax        1.0
ptratio    1.0
b          1.0
lstat      1.0
dtype: float64
crim       0.763473
zn         1.050736
indus      1.008171
chas       1.226452
nox        1.054913
rm         0.967565
age        0.974814
dis        0.965007
rad        1.095531
tax        1.040803
ptratio    1.051794
b          1.120162
lstat      0.964901
dtype: float64


In [ ]:
# check size
print(x_train.shape)

(354, 13)


## Step 5. Neural Network - Sequential

### (1) Define Neural Network

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=13, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

### (2) Compile the keras model

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               1792      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


### (3) Fit the keras model on the dataset

In [ ]:
history = model.fit(x_train, y_train, epochs=100, validation_split=0.05)

Epoch 1/100
11/11 [==============================] - 1s 18ms/step - loss: 569.4130 - mae: 22.0110 - val_loss: 569.9194 - val_mae: 22.0467
Epoch 2/100
11/11 [==============================] - 0s 4ms/step - loss: 501.5987 - mae: 20.4821 - val_loss: 495.0259 - val_mae: 20.4408
Epoch 3/100
11/11 [==============================] - 0s 3ms/step - loss: 415.2375 - mae: 18.4192 - val_loss: 393.2456 - val_mae: 18.0741
Epoch 4/100
11/11 [==============================] - 0s 4ms/step - loss: 301.3902 - mae: 15.4155 - val_loss: 268.8671 - val_mae: 14.6659
Epoch 5/100
11/11 [==============================] - 0s 3ms/step - loss: 182.1163 - mae: 11.4325 - val_loss: 147.5909 - val_mae: 10.2943
Epoch 6/100
11/11 [==============================] - 0s 4ms/step - loss: 88.2711 - mae: 7.4801 - val_loss: 74.0901 - val_mae: 7.0475
Epoch 7/100
11/11 [==============================] - 0s 4ms/step - loss: 52.1455 - mae: 5.5227 - val_loss: 46.9113 - val_mae: 5.3317
Epoch 8/100
11/11 [=============================

### (4) Plot Training Performance (Loss, MAE)

In [ ]:
# plot training loss
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],name='Train'))
fig.add_trace(go.Scattergl(y=history.history['val_loss'],name='Valid'))
fig.update_layout(height=500, width=700,xaxis_title='Epoch',yaxis_title='Loss')
fig.show()

In [ ]:
# plot mean absolute error
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['mae'],name='Train'))
fig.add_trace(go.Scattergl(y=history.history['val_mae'],name='Valid'))
fig.update_layout(height=500, width=700,xaxis_title='Epoch',yaxis_title='Mean Absolute Error')
fig.show() 

### (5) Evaluate Model using Test Data

In [ ]:
mse_testdata, mae_testdata = model.evaluate(x_test, y_test)
print('Mean squared error on test data: ', mse_testdata)
print('Mean absolute error on test data: ', mae_testdata)

5/5 [==============================] - 0s 2ms/step - loss: 18.8924 - mae: 2.7837
Mean squared error on test data:  18.892393112182617
Mean absolute error on test data:  2.783689498901367
